In [ ]:
https://www.kaggle.com/datasets/ted8080/house-prices-and-images-socal

In [1]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"skdrecall","key":"e8e472738dc4dbfa6faa4776eb14579e"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


!kaggle datasets list -s housing


ref                                             title                                 size  lastUpdated                 downloadCount  voteCount  usabilityRating  
----------------------------------------------  ------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
camnugent/california-housing-prices             California Housing Prices           409382  2017-11-24 03:14:59.677000         234709       1442  0.85294116       
schirmerchad/bostonhoustingmlnd                 Boston Housing                        4454  2017-06-11 15:07:11.507000          41834        303  0.8235294        
yasserh/housing-prices-dataset                  Housing Prices Dataset                4740  2022-01-12 18:18:46.587000         122873        408  1.0              
dansbecker/melbourne-housing-snapshot           Melbourne Housing Snapshot          461423  2018-06-05 12:52:24.087000         185282       1649  0.7058824        
justinas/housing

In [3]:
!kaggle datasets download -d ted8080/house-prices-and-images-socal

!unzip house-prices-and-images-socal.zip


Streaming output truncated to the last 5000 lines.
  inflating: socal2/socal_pics/5499.jpg  
  inflating: socal2/socal_pics/55.jpg  
  inflating: socal2/socal_pics/550.jpg  
  inflating: socal2/socal_pics/5500.jpg  
  inflating: socal2/socal_pics/5501.jpg  
  inflating: socal2/socal_pics/5502.jpg  
  inflating: socal2/socal_pics/5503.jpg  
  inflating: socal2/socal_pics/5504.jpg  
  inflating: socal2/socal_pics/5505.jpg  
  inflating: socal2/socal_pics/5506.jpg  
  inflating: socal2/socal_pics/5507.jpg  
  inflating: socal2/socal_pics/5508.jpg  
  inflating: socal2/socal_pics/5509.jpg  
  inflating: socal2/socal_pics/551.jpg  
  inflating: socal2/socal_pics/5510.jpg  
  inflating: socal2/socal_pics/5511.jpg  
  inflating: socal2/socal_pics/5512.jpg  
  inflating: socal2/socal_pics/5513.jpg  
  inflating: socal2/socal_pics/5514.jpg  
  inflating: socal2/socal_pics/5515.jpg  
  inflating: socal2/socal_pics/5516.jpg  
  inflating: socal2/socal_pics/5517.jpg  
  inflating: socal2/socal_pic

In [4]:
!pip install torch torchvision scikit-learn pandas pillow

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image


In [13]:
# Load CSV
df = pd.read_csv('/content/socal2.csv')  # Adjust if yours is named differently
print(df.head())

# Columns: image_id, street, citi, n_citi, bed, bath, sqft, price
tabular_cols = ['bed', 'bath', 'sqft', 'n_citi']
target_col = 'price'




   image_id                 street             citi  n_citi  bed  bath  sqft  \
0         0  1317 Van Buren Avenue  Salton City, CA     317    3   2.0  1560   
1         1         124 C Street W      Brawley, CA      48    3   2.0   713   
2         2        2304 Clark Road     Imperial, CA     152    3   1.0   800   
3         3     755 Brawley Avenue      Brawley, CA      48    3   1.0  1082   
4         4  2207 R Carrillo Court     Calexico, CA      55    4   3.0  2547   

    price  
0  201900  
1  228500  
2  273950  
3  350000  
4  385100  


In [ ]:
# Train/test split
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
print(f"Train size: {len(train_df)}, Val size: {len(val_df)}")




In [ ]:
scaler = StandardScaler()
X_train_tab = scaler.fit_transform(train_df[tabular_cols])
X_val_tab = scaler.transform(val_df[tabular_cols])

In [7]:
# Image transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# Dataset class
class HousingDataset(Dataset):
    def __init__(self, df, tabular_data, transform=None):
        self.df = df.reset_index(drop=True)
        self.tabular_data = tabular_data
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_id = self.df.loc[idx, 'image_id']
        img_path = f"/content/socal2/socal_pics/{img_id}.jpg"
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)

        tabular = self.tabular_data[idx]
        target = self.df.loc[idx, 'price']

        return image, torch.tensor(tabular, dtype=torch.float32), torch.tensor(target, dtype=torch.float32)


In [8]:
train_dataset = HousingDataset(train_df, X_train_tab, transform)
val_dataset = HousingDataset(val_df, X_val_tab, transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [9]:
class MultimodalModel(nn.Module):
    def __init__(self, tabular_dim):
        super(MultimodalModel, self).__init__()
        cnn = models.resnet18(pretrained=True)
        self.cnn = nn.Sequential(*list(cnn.children())[:-1])
        self.tabular = nn.Sequential(
            nn.Linear(tabular_dim, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128)
        )
        self.combined = nn.Sequential(
            nn.Linear(512 + 128, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, images, tabular):
        img_feats = self.cnn(images)
        img_feats = img_feats.view(img_feats.size(0), -1)
        tab_feats = self.tabular(tabular)
        combined = torch.cat([img_feats, tab_feats], dim=1)
        return self.combined(combined).squeeze()


In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MultimodalModel(tabular_dim=X_train_tab.shape[1]).to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0
    for images, tabular, targets in train_loader:
        images, tabular, targets = images.to(device), tabular.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(images, tabular)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item() * images.size(0)

    avg_loss = epoch_loss / len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}] Train Loss: {avg_loss:.4f}")


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


KeyboardInterrupt: 

In [ ]:
model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for images, tabular, targets in val_loader:
        images, tabular = images.to(device), tabular.to(device)
        outputs = model(images, tabular)
        y_true.extend(targets.numpy())
        y_pred.extend(outputs.cpu().numpy())

mae = mean_absolute_error(y_true, y_pred)
rmse = mean_squared_error(y_true, y_pred, squared=False)
print(f"Validation MAE: {mae:.2f}, RMSE: {rmse:.2f}")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Save model
torch.save(model.state_dict(), '/content/drive/MyDrive/multimodal_model.pt')
